# RAG (Retrieval-Augmented Generation) is a technique that allows you to use a knowledge base to answer questions.



In [ ]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.embedder.openai import OpenAIEmbedder
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.lancedb import LanceDb, SearchType

# Create a knowledge base from a PDF
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    # Use LanceDB as the vector database
    vector_db=LanceDb(
        table_name="recipes",
        uri="tmp/lancedb",
        search_type=SearchType.vector,
        embedder=OpenAIEmbedder(model="text-embedding-3-small"),
    ),
)
# Comment out after first run as the knowledge base is loaded
knowledge_base.load(recreate=False)

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    # Add the knowledge base to the agent
    knowledge=knowledge_base,
    show_tool_calls=True,
    markdown=True,
)
agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)

# Streamlit app

In [ ]:
import streamlit as st
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.embedder.openai import OpenAIEmbedder
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.lancedb import LanceDb, SearchType

# Streamlit App
st.title("PDF Knowledge Base Query")

st.sidebar.title("Configuration")
pdf_url = st.sidebar.text_input("PDF URL", value="https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf")
search_type = st.sidebar.selectbox("Search Type", ["vector", "scalar"], index=0)
embedder_model = st.sidebar.text_input("Embedder Model", value="text-embedding-3-small")
table_name = st.sidebar.text_input("Table Name", value="recipes")
db_uri = st.sidebar.text_input("Database URI", value="tmp/lancedb")

if st.sidebar.button("Load Knowledge Base"):
    with st.spinner("Loading knowledge base..."):
        # Create a knowledge base from the PDF
        knowledge_base = PDFUrlKnowledgeBase(
            urls=[pdf_url],
            vector_db=LanceDb(
                table_name=table_name,
                uri=db_uri,
                search_type=SearchType[search_type],
                embedder=OpenAIEmbedder(model=embedder_model),
            ),
        )
        # Load the knowledge base (do not recreate if already exists)
        knowledge_base.load(recreate=False)
        st.success("Knowledge base loaded successfully!")

# Check if knowledge base is ready
if "knowledge_base" not in st.session_state:
    st.warning("Please load the knowledge base first.")
else:
    # Create an agent with the knowledge base
    agent = Agent(
        model=OpenAIChat(id="gpt-4o"),
        knowledge=st.session_state["knowledge_base"],
        show_tool_calls=True,
        markdown=True,
    )

    # Query input
    query = st.text_input("Enter your query:", "")

    if st.button("Submit Query"):
        if query.strip() == "":
            st.warning("Please enter a valid query.")
        else:
            st.subheader("Response")
            with st.spinner("Processing your query..."):
                response = agent.ask(query, stream=False)
                st.markdown(response)
